# 04 - RAG with Memory (Conversational RAG)

**Architecture:** RAG with Conversational Memory

**Complexity:** ⭐⭐

**Use Cases:**
- Chatbots and conversational AI
- Customer support systems  
- Interactive Q&A sessions

**Key Feature:** Maintains chat history to handle follow-up questions and anaphoric references.

**Example:**
```
User: "What is RAG?"
Bot: "RAG is Retrieval-Augmented Generation..."
User: "What are its main components?"  ← References "RAG" from context
Bot: "The main components of RAG are..."  ← Understands reference
```

## 1. Setup

In [ ]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL, DEFAULT_K
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import MEMORY_RAG_PROMPT

print_section_header("Setup: RAG with Memory")

# Load vector store
embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": DEFAULT_K})

# Initialize LLM
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("\n✅ Setup complete!")

## 2. Memory Setup

We'll use `RunnableWithMessageHistory` to add conversational memory to our RAG chain.

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print_section_header("Memory Configuration")

# Session store (in-memory for demo)
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

print("✓ Memory store configured")

# Build base chain
base_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | MEMORY_RAG_PROMPT
    | llm
    | StrOutputParser()
)

# Wrap with memory
conversational_chain = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✓ Conversational chain created with memory")
print("\n💡 Chat history is maintained per session_id")

## 3. Test Conversational Flow

In [ ]:
print_section_header("Conversational Test")

session_id = "user_123"

# First question
print("User: What is RAG?\n")
response1 = conversational_chain.invoke(
    {"input": "What is RAG?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"Bot: {response1}\n")
print("=" * 80)

# Follow-up with anaphoric reference
print("\nUser: What are its main components?\n")
response2 = conversational_chain.invoke(
    {"input": "What are its main components?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"Bot: {response2}\n")
print("=" * 80)

# Another follow-up
print("\nUser: How do I implement it?\n")
response3 = conversational_chain.invoke(
    {"input": "How do I implement it?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"Bot: {response3}")

print("\n✅ Conversation maintained context successfully!")

## 4. View Chat History

In [ ]:
print_section_header("Chat History")

history = store[session_id]
print(f"Messages in session '{session_id}': {len(history.messages)}\n")

for i, msg in enumerate(history.messages, 1):
    role = "User" if msg.type == "human" else "Bot"
    content = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
    print(f"{i}. {role}: {content}\n")

## 5. Comparison: Memory vs No Memory

In [ ]:
from shared.prompts import RAG_PROMPT_TEMPLATE

print_section_header("Comparison: Memory vs No Memory")

# Simple RAG (no memory)
simple_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)

# Test with anaphoric query
query = "What are the advantages of using it?"

print(f"Query (after discussing RAG): '{query}'\n")
print("=" * 80)

print("\n[Simple RAG - NO MEMORY]")
try:
    simple_response = simple_chain.invoke(query)
    print(simple_response[:300])
except Exception as e:
    print(f"Cannot answer: {e}")

print("\n" + "=" * 80)
print("\n[Memory RAG - WITH MEMORY]")
memory_response = conversational_chain.invoke(
    {"input": query},
    config={"configurable": {"session_id": session_id}}
)
print(memory_response[:300])

print("\n" + "=" * 80)
print("\n💡 Memory RAG understands 'it' refers to RAG from conversation context!")

## Summary

### Architecture: RAG with Memory

**Flow:**
```
User Query + Chat History → Retriever → LLM + Prompt → Response
                                            ↓
                                    Update History
```

**Key Components:**
- `RunnableWithMessageHistory`: LCEL wrapper for memory
- `ChatMessageHistory`: Stores conversation
- `MessagesPlaceholder`: Injects history into prompt

**Advantages:**
✅ Handles follow-up questions  
✅ Understands anaphoric references ("it", "that", "them")  
✅ More natural conversations  
✅ Context accumulates over session  

**Limitations:**
- Higher cost (more tokens in context)
- Memory can grow large
- Privacy considerations (stores conversations)

**Production Tips:**
- Use `ConversationBufferWindowMemory` to limit history size
- Implement conversation summarization for long sessions
- Store sessions in database (Redis, PostgreSQL)
- Add conversation timeout/expiry

**Next:** [05_branched_rag.ipynb](05_branched_rag.ipynb) - Multi-query parallel retrieval